In [1]:
!pip install -r requirements.txt -q


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\52477\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Setting

In [2]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image
import tempfile
import subprocess
from pydub import AudioSegment
import time
from typing import List, Tuple
import anthropic
from docx import Document
import aspose.words as aw

C:\Users\52477\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load environment variables from .env file
load_dotenv()

# Retrieve OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins with {openai_api_key[:8]}")
else:
    print("OpenAI API Key is not set")

# Initialize OpenAI client
MODEL = "gpt-4o-mini"
openai = OpenAI()

# Retrieve Anthropic (Claude) API key
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
claude_ = anthropic.Anthropic()

OpenAI API Key exists and begins with sk-proj-


## Definition tools

In [4]:
def write_docs(new_name: str, date: str, destination: str, price: str) -> None:
    """
    Generates a flight reservation confirmation document in Word format and converts it to PDF.

    This function takes flight reservation details and updates a template file 
    ('FormatoConfirmacionReserva.docx') with the provided information, generating a new 
    document with the specified name and exporting it to PDF.

    Args:
    -----
    new_name : str
        The base name for the output file (without extension).
    date : str
        The reservation date in the format 'DD de MMMM'.
    destination : str
        The name of the destination city in lowercase and without accents.
    price : str
        The ticket price, formatted as a string (e.g., '$799').

    """

    # Load the Word document template
    doc = Document("FormatoConfirmacionReserva.docx")

    # Capitalize the first letter of the destination city
    destination = destination[0].upper() + destination[1:]

    # Update the table with reservation details
    doc.tables[0].cell(0, 1).text = date  # Set the reservation date
    doc.tables[1].cell(1, 2).text = destination  # Set the destination city
    doc.tables[1].cell(1, 3).text = price  # Set the ticket price
    doc.tables[1].cell(3, 3).text = price  # Duplicate price in another table field if needed

    # Save the updated document as a Word file
    word_filename = new_name + ".docx"
    doc.save(word_filename)

    # Convert the Word document to PDF
    doc = aw.Document(word_filename)
    doc.save(new_name + ".pdf", aw.SaveFormat.PDF)

    print("PDF successfully generated.")

def transcribe_audio(audio):
    """
    Transcribes an audio file into text using OpenAI's Whisper model.

    This function takes an audio file path, processes it using OpenAI's Whisper-1 model, 
    and returns the transcribed text.

    Args:
    -----
    audio : str
        The file path of the audio file to be transcribed.

    Returns:
    --------
    str:
        - The transcribed text from the audio file if successful.
        - An error message if an exception occurs.
    """
    try:
        # Open the audio file in binary mode
        with open(audio, "rb") as audio_file:
            # Send the audio file to OpenAI's Whisper model for transcription
            transcript = openai.audio.transcriptions.create(
                file=audio_file,
                model="whisper-1"
            )

        print(transcript)  # Print the full transcript object for debugging
        return transcript.text  # Return only the transcribed text
    
    except Exception as e:
        # Return an error message in case of failure
        return f"Error: {str(e)}"

def create_image(city):
    """
    Generates an image representing a vacation in a given city using the DALL·E model.

    This function takes a city name, sends a prompt to OpenAI's DALL·E model to generate a vibrant pop-art 
    style image of the city's tourist attractions, and returns the image as a PIL Image object.

    Args:
    -----
    city : str
        The name of the city for which to generate the image.

    Returns:
    --------
    PIL.Image.Image
        The generated image in pop-art style representing the vacation in the specified city.
    """
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showcasing tourist attractions and everything unique to {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

def play_audio(audio_segment):
    """
    Plays the given audio segment using the ffplay command-line tool.

    This function exports the provided audio segment as a temporary WAV file, 
    plays it using the ffplay tool, and then deletes the temporary file after playback.

    Args:
    -----
    audio_segment : pydub.AudioSegment
        The audio segment to be played.
    
    Returns:
    --------
    None
    """
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) 
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass

def talker(message):
    """
    Converts a given message to speech using OpenAI's text-to-speech model, 
    and plays the generated audio.

    This function sends the input message to OpenAI's API, retrieves the audio response, 
    converts it to an audio segment, and plays the audio using the play_audio function.

    Args:
    -----
    message : str
        The message to be converted into speech.
    
    Returns:
    --------
    None
    """
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)

def format_json_response(json_content: str, language_select: str) -> str:
    """
    Formats a JSON response into a human-friendly text representation.

    This function makes an additional API call to OpenAI's language model to convert raw JSON data 
    into a more readable and user-friendly format. The function ensures that the output is clear 
    and natural, without explicitly mentioning that the data was originally in JSON format.

    Args:
    -----
    json_content : str
        The raw JSON string containing the response data that needs to be formatted.
    language_select : str
        The language preference for formatting (currently not used in the function but can be extended).

    Returns:
    --------
    str:
        A formatted, human-readable version of the JSON response.
    """
    try:

        system_message = (
            "Eres un asistente útil para una aerolínea llamada FlightAI."
            "Da respuestas breves y corteses, de no más de una oración. "
            "Formatea las respuestas JSON en texto natural. "
            "Presenta la información de manera clara y amigable. "
            "No menciones que estás formateando JSON y tampoco que el texto se tradujó."
        )
        
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": f"Formatea esta información de manera amigable: {json_content}"}
        ]
        
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error formateando JSON: {str(e)}")
        return json_content  # Devuelve el contenido original si hay error

In [5]:
# Dictionary of ticket prices for different destinations
ticket_prices = {
    "oaxaca": "$799",
    "paris": "$899",
    "tokyo": "$1400",
    "berlin": "$499",
    "new york": "$650",
    "madrid": "$750",
    "londres": "$950",
    "buenos aires": "$580",
    "sydney": "$1600",
    "toronto": "$720"}
    
def get_catalogue_flights() -> List[str]:
    """
    Retrieve a list of available flight destinations.

    Returns:
        List[str]: A list of available cities for which ticket prices are provided.
    """
    return list(ticket_prices.keys())

def get_ticket_price(destination_city: str) -> str:
    """
    Retrieve the ticket price for a given destination.

    Args:
        destination_city (str): The city for which the ticket price is requested.

    Returns:
        str: The ticket price as a string (e.g., "$799") or "Unknown" if the city is not in the catalog.
    """
    return ticket_prices.get(destination_city.lower(), "Unknown")  # Ensure case insensitivity

def get_reservation(want_reservation: bool, date_reservation: str, destination_city: str) -> Tuple[str, str]:
    """
    Process a flight reservation request.

    Args:
        want_reservation (bool): Whether the user wants to make a reservation.
        date_reservation (str): The requested reservation date.
        destination_city (str): The destination city.

    Returns:
        Tuple[str, str]: A confirmation message and the ticket price, or "Unknown" if the reservation fails.
    """
    city = destination_city.lower()  # Normalize input for case consistency
    price = ticket_prices.get(city, "Unknown")

    if price != "Unknown" and want_reservation:
        print(f"Reservation stored for {date_reservation}")  # Log reservation date
        return "Ticket reserved", price
    return "Unknown", "Unknown"
        
def get_translation(text: str, target_language: str) -> str:
    """
    Translate a given text into the specified target language.

    Args:
        text (str): The text to be translated.
        target_language (str): The target language for translation.

    Returns:
        str: The translated text.
    """
    system_message: str = (
        """Eres un asistente experto en traducción. 
        Responde únicamente con el texto traducido en el idioma solicitado,
        sin añadir explicaciones ni mencionar el proceso de traducción."""
    )

    messages = [
        {'role': 'user', 'content': text},
        {'role': 'user', 'content': f"Translate to {target_language}"}
    ]

    # Request translation from Claude model
    response = claude_.messages.create(
        model="claude-3-haiku-20240307",
        system=system_message,
        messages=messages,
        max_tokens=200
    )

    return response.content[0].text  # Extract translated text from response

In [6]:
# Dictionary defining the function that retrieves available flight destinations
catalogue_flights_function = {
    "name": "get_catalogue_flights",
    "description": """Devuelve una lista con los destinos disponibles en la aerolínea.
                   Este catálogo incluye todas las ciudades a las que se ofrecen vuelos. 
                   Los usuarios pueden solicitar esta información de distintas maneras, como: 
                   '¿A qué destinos vuelan?', 'Lista de vuelos disponibles', 'Ciudades con vuelos en la aerolínea', 
                   "'Opciones de viaje disponibles', '¿A qué lugares puedo volar desde aquí?'.""",
    "parameters": {
        "type": "object",
        "properties": {},  
        "required": []   
    }
}

# Dictionary defining the function that retrieves ticket prices
price_function = {
    "name": "get_ticket_price",
    "description": "Obtén el precio de un billete de ida y vuelta a la ciudad de destino. Llámalo siempre que necesites saber el precio del billete, por ejemplo, cuando un cliente pregunte '¿Cuánto cuesta un billete a esta ciudad?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "La ciudad a la que el cliente desea viajar",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# Dictionary defining the function that processes flight reservations
reservation_function = {
    "name": "get_reservation",
    "description": "Generar un voucher de reserva solo cuando el cliente haya proporcionado explícitamente todos los detalles del vuelo (destino, fecha, etc.). No asumas información ni completes datos automáticamente. Si el cliente menciona una intención de compra sin dar detalles específicos, primero debes preguntarle por la información faltante antes de proceder. Ejemplo: Si el cliente dice 'Quiero comprar un boleto de avión', primero pregunta '¿Para qué destino y en qué fecha?' y solo genera el voucher cuando haya confirmado todos los datos.",
    "parameters": {
        "type": "object",
        "properties": {
            "want_reservation": {
                "type": "boolean",
                "description": "Si el cliente quiere comprar un ticket de avión (true) o no (false).",
            },
            "date_reservation":{
                "type": "string",
                "description": "Día y mes de la reserva en formato 'DD de MMMM'. Ejemplo: '13 de marzo'."
            },
            "destination_city": {
                "type": "string",
                "description": "La ciudad a la que el cliente desea viajar en minusculas y sin acentos",
            },

        },
        "required": ["want_reservation", "date_reservation", "destination_city"],
        "additionalProperties": False
    }
}

# Dictionary defining the function that handles text translation
translation_function = {
    "name": "get_translation",
    "description": "traducir a otro idioma el contenido",
    "parameters": {
        "type": "object",
        "properties": {
            "message": {
                "type": "string",
                "description": "El mensaje a traducir",
            },
            "language": {
                "type": "string",
                "description": "El idioma al que se debe traducir el mensaje"
            }
        },
        "required": ["message", "language"],
        "additionalProperties": False
    }
}

In [7]:
tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": reservation_function},
         {"type": "function", "function": translation_function},
         {"type": "function", "function": catalogue_flights_function}]

## Handling tools

In [8]:
system_message = "Eres un asistente útil para una aerolínea llamada FlightAI. "
system_message += "Da respuestas breves y corteses, de no más de una oración. "
system_message += "Se siempre preciso. Si no sabes la respuesta, dilo."

In [9]:
def handle_tool_call(message, language_select):
    """
    Processes tool function calls based on the user's message and selected language.

    This function identifies the type of tool call (e.g., translation, reservation, ticket price),
    extracts the relevant arguments, calls the corresponding functions, and prepares the response
    in the appropriate format. It also handles any errors that may occur during the process.

    Args:
    -----
    message : object
        The incoming message object that contains tool calls and associated arguments.
    language_select : str
        The language selected for translation or other language-related tasks.

    Returns:
    --------
    tuple: A tuple containing:
        - response (dict): The response message with tool results.
        - city (str): The destination city for the reservation or price query (if applicable).
    """
    tool_call = message.tool_calls[0]  # Extract the first tool call from the message
    function_name = tool_call.function.name  # Get the name of the function being called
    arguments = json.loads(tool_call.function.arguments)  # Parse the arguments in JSON format
    
    result = None  # Placeholder for the result of function calls
    city = None  # Placeholder for the city related to reservation or price
    
    try:
        # Handle translation function
        if function_name == "get_translation":
            texto = arguments.get("texto", "")  # Get the text to translate
            idioma_destino = language_select  # Use the selected language for translation
            result = get_translation(texto, idioma_destino)  # Call the translation function
            response = {
                "role": "tool",
                "content": json.dumps({
                    "translation": result  # Include the translation result
                }),
                "tool_call_id": message.tool_calls[0].id  # Include tool call ID for reference
            }
            return response, None  # Return the response with the translation

        # Handle reservation function
        if function_name == "get_reservation":
            confirmation = arguments.get('want_reservation')  # Whether the user wants to reserve
            date = arguments.get('date_reservation')  # The reservation date
            city = arguments.get('destination_city')  # The destination city for the reservation
            confirmation, price = get_reservation(confirmation, date, city)  # Call reservation function
            response = {
                "role": "tool",
                "content": json.dumps({
                    "confirmation": confirmation  # Include reservation confirmation result
                }),
                "tool_call_id": message.tool_calls[0].id  # Tool call ID
            }
            write_docs("ConfirmationTicket", date, city, str(price))  # Generate confirmation document
            img = create_image(city) # Generate image from the city
            return response, img  # Return the response with the reservation result

        # Handle ticket price query function
        elif function_name == "get_ticket_price":
            city = arguments.get('destination_city')  # Get the destination city for the price query
            price = get_ticket_price(city)  # Get the ticket price for the city
            response = {
                "role": "tool",
                "content": json.dumps({
                    "destination_city": city,  # Include the city name
                    "price": price  # Include the ticket price
                }),
                "tool_call_id": message.tool_calls[0].id  # Tool call ID
            }
            return response, None  # Return the response with the price information

        # Handle catalogue flights function
        elif function_name == "get_catalogue_flights":
            catalogue = get_catalogue_flights()  # Get the available flight catalogue
            response = {
                "role": "tool",
                "content": json.dumps({
                    "catalogue": catalogue  # Include the list of available destinations
                }),
                "tool_call_id": message.tool_calls[0].id  # Tool call ID
            }
            return response, None  # Return the response with the catalogue

    except Exception as e:
        print(f"Error in handle_tool_call: {str(e)}")  # Log the error if one occurs
        return {
            "role": "assistant",
            "content": f"Error processing the function: {str(e)}"  # Return error message
        }, None

In [10]:
def chat(history, language_select):
    """
    Handles a chat interaction with FlightAI, a virtual airline assistant.

    This function processes user messages and generates a response using OpenAI's chat model. 
    It ensures responses are brief, polite, and precise. If a tool call is detected, 
    it delegates the request to the appropriate function and formats the output.

    Args:
    -----
    history : list
        A list of dictionaries representing the conversation history.
        Each dictionary contains a role ("user" or "assistant") and a "content" field.
    language_select : str
        The selected language for the response. If different from Spanish ("es"),
        the response will be translated accordingly.

    Returns:
    --------
    tuple:
        - Updated conversation history with the assistant's response appended.
        - An optional image (currently always `None`).
    """
   
    # Remove metadata from the conversation history if present
    [dic.pop("metadata", None) for dic in history]

    # Construct the message history, including the system prompt
    messages = [{"role": "system", "content": system_message}] + history
    try:
        # Request a response from the OpenAI API
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )
        
        image = None

        # Check if the model's response requires a tool call
        if response.choices[0].finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_response, image = handle_tool_call(message, language_select)
            
            if tool_response:
                messages.append(message)
                messages.append(tool_response)
            
                formatted_content = format_json_response(tool_response["content"], language_select)

                if language_select != "es":
                    translated_content = get_translation(formatted_content, language_select)
                    reply = translated_content
                else:
                    reply = formatted_content
            
            
        else:
            # Process normal assistant response (not requiring a tool call)
            content = response.choices[0].message.content
            if language_select != "es":
                reply = get_translation(content, language_select)
            else:
                reply = content
        
        # Append the assistant's response to the conversation history
        history.append({"role": "assistant", "content": reply})
        talker(reply)
        return history, image
        
    except Exception as e:
        print(f"Error en chat: {str(e)}")
        return history + [{"role": "assistant", "content": "Lo siento, hubo un error procesando tu solicitud."}], None

# Interface

In [11]:
# Define the UI using Gradio Blocks
with gr.Blocks() as ui:
    
    # Create a row with a chatbot and an image display
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")  # Chatbot interface
        image_output = gr.Image(height=500)  # Placeholder for image output (if applicable)
    
    # Dropdown menu to select the language
    with gr.Row():
        language_select = gr.Dropdown(
            choices=["ingles", "español", "francés"],  # Available languages
            label="Selecciona idioma",  # Label for the dropdown
            value="español"  # Default language selection
        )

    # User input fields: text chat and audio upload
    with gr.Row():
        entry = gr.Textbox(label="Chatea con nuestro Agente de IA:")  # Text input field for chat
        audio_input = gr.Audio(type="filepath", label="Sube un archivo de audio")  # Audio upload field

    # Automatically transcribe uploaded audio and place the text into the entry box
    audio_input.change(transcribe_audio, inputs=audio_input, outputs=entry)

    # Clear chat button
    with gr.Row():
        clear = gr.Button("Clear")  # Button to reset chat history

    # Function to process user input and update chat history
    def do_entry(message, history, language_select):
        """
        Handles user messages by appending them to the chat history.

        Args:
        -----
        message : str
            The user's message.
        history : list
            The existing chat history.
        language_select : str
            The selected language for responses.

        Returns:
        --------
        tuple:
            - Empty string (to clear the input field).
            - Updated chat history.
            - Language selection (unchanged).
        """
        history += [{"role": "user", "content": message}]  # Append user message to history
        return "", history, language_select  # Clear input field and return updated values

    # Set up event handling for user input submission
    entry.submit(
        do_entry,  # Function to handle input processing
        inputs=[entry, chatbot, language_select],  # Inputs for processing
        outputs=[entry, chatbot, language_select]  # Outputs to update UI
    ).then(
        chat,  # Call the chat function after processing user input
        inputs=[chatbot, language_select],  # Pass chat history and language selection
        outputs=[chatbot, image_output]  # Update chat history and image output
    )

    # Event to clear the chat history when the "Clear" button is clicked
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

# Launch the Gradio interface in a web browser
ui.launch(inbrowser=True)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\52477\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.41.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Transcription(text='I would like to buy a ticket.')
Reservation stored for 20 de junio
PDF successfully generated.
